In [2]:
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
import json
import pandas as pd
import os
from nltk.stem import WordNetLemmatizer
import numpy as np

/Users/mika/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
schema = {"url", "archive", "title", "date", "text", "summary", "compression", "coverage", "density", "compression_bin", "coverage_bin", "density_bin"}

In [6]:
#todo - nie działa
records = []
for file_name in os.listdir("data"):
  try:
    path = os.path.join("data", file_name)

    with open(path, 'r') as file:
        data = file.read()
        for line in data.splitlines():
            record = json.loads(line)
            try:
                if isinstance(record, dict) and schema.issubset(record.keys()):
                  record['text'] = record['text'].encode('utf-8').decode('utf-8', errors='replace')
                  record['summary'] = record['summary'].encode('utf-8').decode('utf-8', errors='replace')
                  try:
                    records.append(record)
                    pd.DataFrame(records)
                  except TypeError:
                     continue
            except json.JSONDecodeError:
                continue
  except:
    continue

data = pd.DataFrame(records)
data

In [ ]:
data = data[['text', 'summary']]
data

In [ ]:
data.describe()

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)
test_data.to_csv('test_data.csv')
val_data.to_csv('val_data.csv')

In [ ]:
train_data.drop_duplicates()

In [ ]:
text_len = train_data['text'].apply(len)
summary_len = train_data['summary'].apply(len)

print(f"Text:\n min: {text_len.min()}\n max: {text_len.max()}\n mean: {text_len.mean()}")
print(f"Summary:\n min: {summary_len.min()}\n max: {summary_len.max()}\n mean: {summary_len.mean()}")


In [ ]:
train_data = train_data[train_data['summary'].apply(len) <= 200]
train_data

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
train_data_tokens = pd.DataFrame()
train_data_tokens['text'] = train_data['text'].apply(word_tokenize)
train_data_tokens['summary'] = train_data['summary'].apply(word_tokenize)
train_data_tokens

In [ ]:
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
train_data_stopwords = pd.DataFrame()

train_data_stopwords['text'] = train_data_tokens['text'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])
train_data_stopwords['summary'] = train_data_tokens['summary'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])

train_data_stopwords

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
train_data_lemmatize = pd.DataFrame()
nltk.download('wordnet')

train_data_lemmatize['text'] = train_data_stopwords['text'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
train_data_lemmatize['summary'] = train_data_stopwords['summary'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])

train_data_lemmatize


In [ ]:
train_data_lemmatize.to_csv('train_data.csv')